# هدف : کم و زیاد کردن نور و صدا در اوبونتو با حرکات دست
## دست راست برای صدا  و دست چپ برای نور صفحه نمایش

In [1]:
import cv2 
import numpy as np 
import mediapipe as mp
from datetime import datetime

2024-08-25 19:48:41.951985: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 19:48:41.962493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 19:48:41.975266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 19:48:41.978696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 19:48:41.988558: I tensorflow/core/platform/cpu_feature_guar

# نادیده گرفتن هشدار های کم اهمیت

In [2]:
import warnings
from google.protobuf import message as pb_message

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')


# توابع مربوط به صدا

In [3]:
import subprocess

def set_volume_increase(amount):
    subprocess.run(['pactl', 'set-sink-volume', '@DEFAULT_SINK@', f'+{amount}%'])

def set_volume_decrease(amount):
    subprocess.run(['pactl', 'set-sink-volume', '@DEFAULT_SINK@', f'-{amount}%'])

def set_volume(level):
    subprocess.run(['pactl', 'set-sink-volume', '@DEFAULT_SINK@', f'{level}%'])

def get_volume():
    output = subprocess.check_output(['pactl', 'get-sink-volume', '@DEFAULT_SINK@']).decode('utf-8')
    volume = output.split()[4].replace('%', '')
    return int(volume)

# توابع مربوط به نور صفحه نمایش


In [4]:

def get_current_brightness():
    # مسیر فایل‌های سیستمی مرتبط با روشنایی
    brightness_file = "/sys/class/backlight/intel_backlight/brightness"
    max_brightness_file = "/sys/class/backlight/intel_backlight/max_brightness"

    # خواندن مقدار فعلی روشنایی
    with open(brightness_file, 'r') as f:
        current_brightness = int(f.read().strip())

    # خواندن حداکثر مقدار روشنایی
    with open(max_brightness_file, 'r') as f:
        max_brightness = int(f.read().strip())

    # محاسبه درصد روشنایی فعلی
    brightness_percentage = (current_brightness / max_brightness) * 100

    return brightness_percentage

# چاپ درصد روشنایی فعلی
# current_brightness_percentage = get_current_brightness()
# print(f"Current brightness: {current_brightness_percentage:.2f}%")



def set_brightness(level):
    # اطمینان حاصل کنید که مقدار روشنایی بین 0 و 1 است
    if level < 0 or level > 1:
        raise ValueError("Brightness level must be between 0 and 1.")
    
    # مسیر فایل تنظیم روشنایی
    brightness_file = "/sys/class/backlight/intel_backlight/brightness"
    max_brightness_file = "/sys/class/backlight/intel_backlight/max_brightness"

    # خواندن حداکثر مقدار روشنایی
    with open(max_brightness_file, 'r') as f:
        max_brightness = int(f.read().strip())
    
    # محاسبه مقدار جدید روشنایی
    new_brightness = int(max_brightness * level)

    # استفاده از subprocess برای اجرای دستور با sudo
    command = f"echo {new_brightness} | sudo tee {brightness_file} > /dev/null"
    subprocess.run(command, shell=True, check=True)

# تعریف رنگ

In [5]:
# opencv تعریف رنگ‌ها برای استفاده در ترسیمات البته برعکس چون 
red = (0, 0, 255)
green = (0, 255, 0)
blue = (255, 0, 0)

In [6]:
# 1-  Mediapipe راه‌اندازی مدل تشخیص دست از
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 2- Mediapipe راه‌اندازی ابزارهای ترسیم از
mp_drawing = mp.solutions.drawing_utils


I0000 00:00:1724602723.901746   28390 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1724602723.903601   28794 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:

# راه‌اندازی وبکم
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 3- پردازش فریم برای تشخیص دست‌ها
    results = hands.process(frame_rgb)
    
    landmark_4 = []  # تعریف لیست برای نقطه شماره 4
    landmark_8 = []  # تعریف لیست برای نقطه شماره 8
    volume=get_volume()
    brightness = get_current_brightness()
    
    # بررسی اگر دست‌ها تشخیص داده شدند
    if results.multi_hand_landmarks:
        # بررسی اگر دست راست بود
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = handedness.classification[0].label
            
            for id, pos_id in enumerate(hand_landmarks.landmark): # id : شماره نقظه در انگشت ها | pos_id : مختصات نسبی در صفحه
                
                    # برای کار کردن با فاصله بین انگشتان
                    h, w, c = frame.shape  # گرفتن ابعاد تصویر
                    x, y = int(pos_id.x * w), int(pos_id.y * h)  # تبدیل مختصات نسبی به پیکسلی
                    
                    # landmark_list.append([id, x, y])

                    # اگر نقطه شماره 4 (نوک انگشت شست) باشد، دایره‌ای رسم کن
                    if id == 4:
                        cv2.circle(frame, (x, y), 15, blue, -1)  # -1 یعنی دایره توپر باشد
                        landmark_4.append([x,y])

                    # اگر نقطه شماره 4 (نوک انگشت شست) باشد، دایره‌ای رسم کن
                    if id == 8:
                        cv2.circle(frame, (x, y), 15, blue, -1)  # -1 یعنی دایره توپر باشد
                        landmark_8.append([x,y])
                                        
                    # محاسبه فاصله بین نقاط شماره 4 و 8
                    if len(landmark_4) > 0 and len(landmark_8) > 0:
                        x1, y1 = landmark_4[0]
                        x2, y2 = landmark_8[0]
                        distance_between_4_8 = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                        # print(f"Distance between points 4 and 8: {distance_between_4_8}")
                        

            # بررسی اگر دست راست باشد
            if hand_label == 'Right':  
                
                min_distance = 25
                max_distance = 280
                
                min_sound = 0
                max_sound = 100
                
                if distance_between_4_8 < min_distance:
                    distance_between_4_8 = min_distance
                elif distance_between_4_8 > max_distance:
                    distance_between_4_8 = max_distance
                        
                volume = min_sound + (int((distance_between_4_8 - min_distance) * (max_sound - min_sound) / (max_distance - min_distance)))
                # print(volume)
                                    
                # تنظیم صدا بر اساس محاسبه جدید
                set_volume(volume)  # این خط را فعال کنید تا صدا بر اساس فاصله تنظیم شود
                
                # نوشتن اندازه صدا در لحظه
                cv2.putText(frame, f'Volume: {volume}%', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    
            elif hand_label=='Left':
                
                # مقادیر حداقل و حداکثر فاصله
                min_distance = 25  # جایگزین با مقدار واقعی حداقل فاصله
                max_distance = 280  # جایگزین با مقدار واقعی حداکثر فاصله

                min_brightness = 0.1
                max_brightness = 1.0

                # اطمینان از اینکه distance_between_4_8 در بازه‌ی مشخص است
                if distance_between_4_8 < min_distance:
                    distance_between_4_8 = min_distance
                elif distance_between_4_8 > max_distance:
                    distance_between_4_8 = max_distance

                # محاسبه روشنایی بر اساس فاصله
                brightness = min_brightness + (int(distance_between_4_8 - min_distance) * (max_brightness - min_brightness) / (max_distance - min_distance))
                set_brightness(brightness)
                cv2.putText(frame, f'Brightness: {brightness * 100:.0f}%', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
            # رسم اتصالات و نقاط کلیدی دست
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


    # نمایش فریم پردازش شده
    cv2.imshow('Webcam', frame)
    
    # به‌دست آوردن زمان کنونی
    current_time = datetime.now()
    current_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

    k = cv2.waitKey(1) & 0xFF 
    if k==27:    # برای خروج Esc کلید 
        break
    
    elif k== ord('q'): # q یا دکمه 
        break
    
    elif k == ord('s'):
        filename = f'Data/Volume_Hand_Control{current_time}.jpg'
        cv2.imwrite(filename,frame)
        print(f"Image saved as {filename}")
        
# آزاد کردن منابع
cap.release()
cv2.destroyAllWindows()


W0000 00:00:1724602723.922244   28782 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724602723.938968   28789 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


<img src = 'Data/Volume_Hand_Control_1.png'>
<img src = 'Data/Volume_Hand_Control_2.png'>
<img src = 'Data/Volume_Hand_Control_3.png'>
<img src = 'Data/Volume_Hand_Control_4.png'>
<img src = 'Data/Volume_Hand_Control_5.png'>